# Gender Disparity in IMDB Ratings

Do you remember when they announced Ghostbusters was being remade with female leads? People seemed pretty upset about it. Almost immediately, after release, the films rating tanked. Was it a bad movie? Maybe. But a quick look at IMDB's ratings (see here: https://www.imdb.com/title/tt1289401/ratings?demo=imdb_users) shows that although 17% of raters gave it 1 star, 11% of users gave it ten stars. This type of disparity is pretty uncommon - ratings don't usually follow this barbell pattern. Digging deeper I realized that about 24% of women gave 10 stars but about 20% of all men gave 1 star. Because there were three times more men rating the film than women, ratings from men had a much more significant impact. This made me wonder if there are other films where there's a rating disparity between men and women. 

In [185]:
#Download packages
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import math

To start, I put together a big list of movies for the analysis. If you're curious about how I generated the data or want to create your own database, check out my github functions here: https://github.com/sampurkiss/movie_ratings_imdb/blob/master/get_rating_data.py

In [186]:
#Download the data
movie_graph_data = pd.read_csv('https://raw.githubusercontent.com/sampurkiss/movie_ratings_imdb/master/movie_ratings.csv')
#Add additional columns to database    
movie_graph_data['gender_ratings'] = movie_graph_data['no_of_male_ratings']+movie_graph_data['no_of_female_ratings']
# Need above since the total number of ratings given by IMDB doesn't always have gender data associated with it
movie_graph_data['ratings_differential'] = movie_graph_data['males']- movie_graph_data['females']


To start, I was curious if there were a lot of movies where there was a significant difference in how genders rated the film. The below chart shows what each films rating was by respective gender proportion. If all genders rate movies the same, then all movies ratings should fall along the diagonal line. For example, 37% of all men that rated the film Dangal and 37% of all women who rated the film Dangal gave 10 stars. This is the sort of symmetry we would expect, but as you can see, a lot of films fall far from the diagonal line indicating a skewed preference. This isn't particularly surprising, some films tend to target women audiences and some tend to target male audiences. If you poke around in the chart, you'll probably notice a few films that skew higher to one gender that you may not have expected.

Before we go any further, you've probably noticed The Promise is an outlier in a lot of these graphs. I won't discuss it here, but I would encourage you to look into that particular films controversy and speculate on why it's such an outlier. 

In [187]:
hover_text =[]
for i in range(0,len(movie_graph_data)):
    hover_text.append(('Movie: {movie}<br>'+
                       'Release date: {date}<br>'+
                       'Genre: {genre}').format(
                               movie = movie_graph_data['name'].iloc[i],
                               date = int(movie_graph_data['year'].iloc[i]),
                               genre=movie_graph_data['genre'].iloc[i]))
traces =[]
for i in range(1,11):
    trace0 =go.Scatter(x = movie_graph_data['male_rating_'+str(i)]/movie_graph_data['no_of_male_ratings'],
                   y = movie_graph_data['female_rating_'+str(i)]/movie_graph_data['no_of_female_ratings'],
                   mode = 'markers',
                   text = hover_text,
                   name = str(i)+' stars')
    traces.append(trace0)

dissecting_line = go.Scatter(x =[0,2],
                            y = [0,2],
                            mode = 'lines',
                            name = 'Intersecting Line',
                            marker = dict(color = 'rgba(106,114,119,0.5)'))
traces.append(dissecting_line)

layout = go.Layout(title = 'Movie Ratings by Gender Proportion',
                   hovermode = 'closest',
                   showlegend = True,
                   xaxis = dict(title = 'Proportion of All Male Ratings',
                                range=[0, 1],
                               tickformat='.0%'),
                   yaxis = dict(title = 'Proportion of All Female Ratings',
                                range=[0, 1],
                               tickformat='.0%'))
fig = go.Figure(data = traces, layout = layout)

py.iplot(fig, filename='movie-ratings-by-gender')


The first thing I noticed is that IMDB 10 ratings seem to skew female, and 1 ratings seem to skew slightly male. That should mean that movies with more female ratings should have much higher ratings. The chart below shows that women do tend to give significantly more ten ratings and men tend to give ratings between 6 and 8. Women actually give slightly more 1s indicating that the above chart is skewed by a handful of outliers. 

In [188]:
###########################################################
#Ratings histogram
###########################################################

ratings = movie_graph_data[['male_rating_10', 'male_rating_9', 'male_rating_8', 'male_rating_7',
       'male_rating_6', 'male_rating_5', 'male_rating_4', 'male_rating_3',
       'male_rating_2', 'male_rating_1', 'female_rating_10', 'female_rating_9',
       'female_rating_8', 'female_rating_7', 'female_rating_6',
       'female_rating_5', 'female_rating_4', 'female_rating_3',
       'female_rating_2', 'female_rating_1']].aggregate(sum)
ratings = pd.DataFrame(ratings)
ratings.reset_index(inplace=True)
ratings = ratings.rename(columns={'index':'type',0:'number'})

rating_range = []
for i in range(10, 0, -1):
    rating_range.append(i)

trace = go.Bar(x = rating_range,
                     y = ratings['number'].iloc[0:10]/sum(ratings['number'].iloc[0:10]),
                     name = "Mens")
trace2 = go.Bar(x = rating_range,
                     y = ratings['number'].iloc[10:20]/sum(ratings['number'].iloc[10:20]),
                     name = "Womens")
    
layout = go.Layout(title='Ratings distribution by gender',
                  yaxis=dict(title='Proportion of Respective Gender',
                            tickformat='.0%'),
                  xaxis=dict(title='Star Rating Out of 10')) 
    
fig = go.Figure(data = [trace, trace2], layout=layout)
py.iplot(fig, filename = 'rating-distribution')



This should be great news for aspiring film makers - create films for female audiences and your rating will soar! Unfortunately, a challenge is that IMDB ratings are dominated by men. That means male opinions are more likely to sway IMDB ratings. 

In [189]:
x0 = movie_graph_data['no_of_female_ratings']/movie_graph_data['gender_ratings']
data = go.Histogram(x = x0)

layout = go.Layout(title = 'Proportion of women giving ratings',
                   xaxis = dict(title = 'Proportion of Female Raters',
                                range = [0,1],
                               tickformat='.0%'),
                  yaxis=dict(title='Number of Movies'))

fig = go.Figure(data = [data], layout = layout)

py.iplot(fig, filename='proportion-of-female-raters')

So lets dig in to these gender differences a little more. For each film, I ranked the IMDB male rating and the IMDB female rating to get an order by each gender. I then took the difference to compare how genders ranked the movies. For example, if the male ranking of a film was 300 (as in 300th best rated movie according to males) and the female ranking of the same film was 400, then the differential would be 100. A higher differential indicates a larger difference between how females and males rated a movie. As shown below, the larger the amount of females as a proportion of total that rated each movie, the wider the differential becomes. This means that the less women there are that rated a movie, the more likely it is that men rated it highly. 

In [191]:
#Get ranking differentials
differentials = movie_graph_data
differentials['male_ranking']= differentials['males'].rank()
differentials['female_ranking']= differentials['females'].rank()
differentials['ranking_differential'] = (differentials['male_ranking'] -
                                        differentials['female_ranking'])

trace = go.Scatter(y= differentials['ranking_differential'],
                  x = differentials['no_of_female_ratings']/differentials['gender_ratings'],
                  text = differentials['name'],
                  mode='markers',
                  name ='')

layout = go.Layout(title = 'Ranking differential by proportion of films rated by women',
                   hovermode = 'closest',
                   showlegend = False,
                  xaxis = dict(tickformat='.0%',
                              title = 'Proportion of Female Ratings'),
                  yaxis=dict(tickformat='0,000',
                            title='Ranking differential'))

fig = go.Figure(data = [trace], layout = layout)

py.iplot(fig, filename = 'rating-differential')

My first thought was, well, that's a big list of movies, maybe there's just a lot of noise. IMDB is pretty big on its top 250 movies so I thought maybe I should restrict my dataset to just that. To get to its top 250, IMDB requires that each film has at least 25,000 ratings. I restricted my dataset to only those with ratings above 25,000 and ran the same ranking process. Unfortunately, that didn't clear the issue. Even for each genders respective top 250, the more female raters, the wider the gap between gender rankings. 

In [192]:
differentials = movie_graph_data
differentials = differentials.loc[differentials['no_of_ratings']>25000] #Criteria here https://help.imdb.com/article/imdb/track-movies-tv/faq-for-imdb-ratings/G67Y87TFYYP6TWAV#
differentials['male_ranking']= differentials['males'].rank()
differentials['female_ranking']= differentials['females'].rank()
differentials = differentials.query("male_ranking<250|female_ranking<250")
differentials['ranking_differential'] = (differentials['male_ranking'] -
                                        differentials['female_ranking'])


trace = go.Scatter(y= differentials['ranking_differential'],
                  x = differentials['no_of_female_ratings']/differentials['gender_ratings'],
                  text = differentials['name'],
                  mode='markers',
                  name ='')

layout = go.Layout(title = 'Ranking differential by proportion of films rated by women',
                   hovermode = 'closest',
                   showlegend = False,
                  xaxis = dict(tickformat='.0%',
                              title = 'Proportion of Female Ratings'),
                  yaxis=dict(tickformat='0,000',
                            title='Ranking differential'))

fig = go.Figure(data = [trace], layout = layout)

py.iplot(fig, filename = 'rating-differential')

To look at how these ranking differentials show up in actual ratings, I took the ratings differential (male rating - female rating) to find what the average difference is by proportion of female raters. Consistent with above, the larger the proportion of females rating a movie, the wider the gap between male and female ratings. 

In [193]:
ratings_diff = movie_graph_data[['no_of_female_ratings','gender_ratings','ratings_differential']].copy()
ratings_diff.loc[:,'proportion'] = (ratings_diff.loc[:,'no_of_female_ratings']/ratings_diff.loc[:,'gender_ratings']/5).round(2)*5
ratings_diff = (ratings_diff
                .groupby(['proportion'])[['proportion','ratings_differential']]
                .agg(['mean','count']))
ratings_diff.loc[:,'films'] = ['Number of films: ' + str(ratings_diff['proportion']['count'].iloc[i]) for i in range(0,len(ratings_diff))]

trace = go.Scatter(x = ratings_diff['proportion']['mean'],
                   y = ratings_diff['ratings_differential']['mean'],
                   mode = 'lines',
                   text = ratings_diff['films'],
                   name = '')

layout = go.Layout(title = 'Ratings differential by proportion of female raters',
                   showlegend = False,
                    xaxis = dict(title = 'Proportion of Female Raters',
                                 range=[0, .75],
                                tickformat ='.0%'),
                       yaxis = dict(title = 'Ratings differential',
                    range=[-2, 2]))
                    
fig= go.Figure(data = [trace], layout = layout)

py.iplot(fig, filename = 'ratings-differentials-female')


So then I thought, well females are more likely to give higher ratings than men, so, in spite of gender differences, movies with more women rating them should have a higher rating. Unfortunately, I was wrong again. I took the average IMDB rating by proportion of films rated by men vs. women and found that the more females that rated a film, the lower the rating was likely to be. 

In [194]:
ratings_diff = movie_graph_data[['no_of_female_ratings','gender_ratings','imdb_users']].copy()
ratings_diff.loc[:,'proportion'] = (ratings_diff.loc[:,'no_of_female_ratings']/ratings_diff.loc[:,'gender_ratings']/5).round(2)*5
ratings_diff = ratings_diff.groupby(['proportion'])[['proportion','imdb_users']].agg(['mean','count'])
ratings_diff.loc[:,'films'] = ['Number of films: ' + str(ratings_diff['proportion']['count'].iloc[i]) for i in range(0,len(ratings_diff))]

trace = go.Scatter(x = ratings_diff['proportion']['mean'],
                   y = ratings_diff['imdb_users']['mean'],
                   mode = 'lines',
                   text = ratings_diff['films'],
                   name = '')

layout = go.Layout(title = 'Total rating by proportion of female raters',
                   showlegend = False,
                    xaxis = dict(title = 'Proportion of Female Raters',
                                 range=[0, .75],
                                tickformat ='.0%'),
                       yaxis = dict(title = 'IMDB Rating',
                    range=[5, 10]))
                    
fig= go.Figure(data = [trace], layout = layout)

py.iplot(fig, filename = 'ratings-female-proportions')


So, now we know that movie ratings are dominated by men, and that there's a wide gap between movies men like and movies women like. As a result, movies that way more women watch tend to have worse ratings than movies that far more men watch. Looking at box office results for films using the same proportion as above, it seems that films do better when they're not at the extreme ends, i.e. when films

In [195]:
###########################################################
#Box office returns for female rated movies
###########################################################
box_office = movie_graph_data
#Get average box office return by rating
box_office['proportion_female'] = (box_office['no_of_female_ratings']/\
                                         box_office['gender_ratings']/5).round(2)*5

#Note: need to filter out films without any box office information
box_office=(box_office[box_office['gross_worldwide']>0][['gross_worldwide','proportion_female']]
            .groupby(by=['proportion_female'])
            .agg(['mean','count'])
            .reset_index())


trace = go.Scatter(x = box_office['proportion_female'],
                   y = box_office['gross_worldwide']['mean'],
                   mode = 'lines',
                   text = ['Number of films: '+str(num) for num in box_office['gross_worldwide']['count']],
                   name = '')


layout = go.Layout(title = 'Average box office amount by proportion of female raters',
                   showlegend = False,
                   hovermode = 'closest',
                    xaxis = dict(title = 'Proportion of Female Raters',
                                 range=[0, .75],
                                tickformat ='.0%'),
                       yaxis = dict(title = 'Box Office Amount'))
                    
fig= go.Figure(data = [trace], layout = layout)

py.iplot(fig, filename = 'box-office-revenue-female')

In [196]:
ratings_diff = movie_graph_data[['no_of_female_ratings','gender_ratings','male_rating_1', 'male_rating_2','male_rating_3']].copy()
ratings_diff.loc[:,'proportion'] = (ratings_diff.loc[:,'no_of_female_ratings']/ratings_diff.loc[:,'gender_ratings']/5).round(2)*5
ratings_diff.loc[:,'ones'] = ((ratings_diff.loc[:,'male_rating_1']+ratings_diff.loc[:,'male_rating_2'])/
                               ratings_diff.loc[:,'gender_ratings'])
ratings_diff = ratings_diff.groupby(['proportion'])[['proportion','ones']].agg(['mean','count'])
ratings_diff.loc[:,'films'] = ['Number of films: ' + str(ratings_diff['proportion']['count'].iloc[i]) for i in range(0,len(ratings_diff))]

trace = go.Scatter(x = ratings_diff['proportion']['mean'],
                   y = ratings_diff['ones']['mean'],
                   mode = 'lines',
                   text = ratings_diff['films'],
                   name = '')

layout = go.Layout(title = 'Total rating by proportion of female raters',
                   showlegend = False,
                    xaxis = dict(title = 'Proportion of Female Raters',
                                 range=[0, .75],
                                tickformat ='.0%'),
                       yaxis = dict(title = 'IMDB Rating',
                    range=[0, .2]))
                    
fig= go.Figure(data = [trace], layout = layout)

py.iplot(fig, filename = 'ratings-differentials-female')
